In [1]:
from ultralytics import YOLO

In [2]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import tqdm
import face_recognition
from numpy.ma.core import ceil

In [3]:
modelSegment = YOLO("D:\DownLoad\ModelSegmentM\FinalMLast.pt") 
modelDetect = YOLO("D:\DownLoad\ModelDetectM\FinalMLast.pt")

In [4]:
color=[np.random.randint(0,255), np.random.randint(0,255), np.random.randint(0,255)] #color to merge with mask in segmentation task

#Real Time Model 


In [9]:
import time 
prevTime=0
#without face regcognition
cap=cv2.VideoCapture(0)
while True:
    ret, frame= cap.read()
    frame=cv2.flip(frame,1)
    frame=cv2.resize(frame,(512,512))
    results=modelSegment.predict(source=frame,conf=0.7, save=False)
    
    for i in range(0,len(results[0])):
        #cv2_imshow((results[0].masks.data[i].numpy()*255).astype("uint8"))
        mask=results[0].masks.cpu().data[i].numpy()
        mask_image = frame.copy()
        mask_image[mask == 0] = [255, 255, 255]
        color_img=frame.copy()
        color_img[mask == 0] = [255, 255, 255]
        color_img[mask == 1] = color
        frame[mask == 1]=cv2.addWeighted(frame[mask == 1],0.5,color_img[mask == 1],0.5,0)
        
        resDetect= modelDetect.predict(mask_image,conf=0.7)
        
        for r in resDetect:
            boxes=r.boxes
            for box in boxes:
              x1,y1,x2,y2= box.xyxy[0]
              x1,y1,x2,y2= int(x1),int(y1),int(x2),int(y2)
              w,h=x1-x2,y1-y2
              conf=ceil(float(box.conf[0])*100)/100
              labelBox=modelDetect.names.get(box.cls.item())
              if(labelBox=='not_helmet' or labelBox =='not_safetyVest'):
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
                cv2.putText(frame,f'{labelBox}  {conf}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,0,255),thickness=1)
              else:
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 1)
                cv2.putText(frame,f'{labelBox}  {conf}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,255,0),thickness=1)
    new_frame_time = time.time() 
    print(f'{round(new_frame_time-prevTime,2)} s')
    prevTime=new_frame_time
    cv2.imshow("Image",frame)
    if cv2.waitKey(10) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 512x512 1 people, 56.8ms
Speed: 0.0ms preprocess, 56.8ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 not_helmet, 1 safetyVest, 47.3ms
Speed: 0.0ms preprocess, 47.3ms inference, 2.5ms postprocess per image at shape (1, 3, 512, 512)
1724806298.62 s

0: 512x512 1 people, 42.8ms
Speed: 1.0ms preprocess, 42.8ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 not_helmet, 1 safetyVest, 35.5ms
Speed: 1.5ms preprocess, 35.5ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)
0.16 s

0: 512x512 1 people, 40.6ms
Speed: 2.0ms preprocess, 40.6ms inference, 5.6ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 not_helmet, 1 safetyVest, 31.2ms
Speed: 2.0ms preprocess, 31.2ms inference, 5.5ms postprocess per image at shape (1, 3, 512, 512)
0.13 s

0: 512x512 1 people, 40.9ms
Speed: 2.2ms preprocess, 40.9ms inference, 6.6ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 not_helmet, 1 safety

In [11]:
import time 
prevTime=0
#without face regcognition
cap=cv2.VideoCapture("D:\DownLoad\VideoDownload\Stay Safe Anywhere- 5 Construction Safety Tips.mp4")
#cap=cv2.VideoCapture(0)
while True:
    ret, frame= cap.read()
    if not ret:
        print("No more frames in the video")
        break
    frame=cv2.flip(frame,1)
    frame=cv2.resize(frame,(512,512))
    results=modelSegment.predict(source=frame,conf=0.65, save=False)
    for i in range(0,len(results[0])):
        #cv2_imshow((results[0].masks.data[i].numpy()*255).astype("uint8"))
        mask=results[0].masks.cpu().data[i].numpy()
        mask = cv2.resize(mask, (512,512), interpolation=cv2.INTER_NEAREST)#test
        mask_image = frame.copy()
        mask_image[mask == 0] = [255, 255, 255]
        color_img=frame.copy()
        color_img[mask == 0] = [255, 255, 255]
        color_img[mask == 1] = color #[np.random.randint(0,255), np.random.randint(0,255), np.random.randint(0,255)]
        frame[mask == 1]=cv2.addWeighted(frame[mask == 1],0.5,color_img[mask == 1],0.5,0)
        
        resDetect= modelDetect.predict(mask_image,conf=0.7)
        
        for r in resDetect:
            boxes=r.boxes
            for box in boxes:
              x1,y1,x2,y2= box.xyxy[0]
              x1,y1,x2,y2= int(x1),int(y1),int(x2),int(y2)
              w,h=x1-x2,y1-y2
              conf=ceil(float(box.conf[0])*100)/100
              labelBox=modelDetect.names.get(box.cls.item())
              if(labelBox=='not_helmet' or labelBox =='not_safetyVest'):
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
                cv2.putText(frame,f'{labelBox}  {conf}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,0,255),thickness=1)
              else:
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 1)
                cv2.putText(frame,f'{labelBox}  {conf}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,255,0),thickness=1)
    
    new_frame_time = time.time() 
    print(f'{round(new_frame_time-prevTime,2)} s')
    prevTime=new_frame_time
    cv2.imshow("Image",frame)
    if cv2.waitKey(10) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 512x512 (no detections), 48.1ms
Speed: 1.0ms preprocess, 48.1ms inference, 2.2ms postprocess per image at shape (1, 3, 512, 512)
1724806367.67 s

0: 512x512 (no detections), 53.2ms
Speed: 4.6ms preprocess, 53.2ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)
0.09 s

0: 512x512 1 people, 56.4ms
Speed: 3.0ms preprocess, 56.4ms inference, 3.2ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 helmet, 1 not_safetyVest, 45.5ms
Speed: 0.0ms preprocess, 45.5ms inference, 3.0ms postprocess per image at shape (1, 3, 512, 512)
0.16 s

0: 512x512 1 people, 42.0ms
Speed: 1.3ms preprocess, 42.0ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 helmet, 34.6ms
Speed: 2.8ms preprocess, 34.6ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 512)
0.15 s

0: 512x512 1 people, 44.0ms
Speed: 2.0ms preprocess, 44.0ms inference, 5.8ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 helmet, 1 safetyVest, 35.2ms
Speed:

In [23]:
#Only Detect Task
#without face regcognition
import time 
prevTime=0
cap=cv2.VideoCapture("d:\DownLoad\VideoDownload\ppe-1.mp4")
#cap=cv2.VideoCapture(0)
while True:
    ret, frame= cap.read()
    if not ret:
        print("No more frames in the video")
        break
    frame=cv2.flip(frame,1)
    frame=cv2.resize(frame,(720,1080))
    resDetect= modelDetect.predict(frame,conf=0.65)
        
    for r in resDetect:
        boxes=r.boxes
        for box in boxes:
            x1,y1,x2,y2= box.xyxy[0]
            x1,y1,x2,y2= int(x1),int(y1),int(x2),int(y2)
            w,h=x1-x2,y1-y2
            conf=ceil(float(box.conf[0])*100)/100
            labelBox=modelDetect.names.get(box.cls.item())
            if(labelBox=='not_helmet' or labelBox =='not_safetyVest'):
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
                cv2.putText(frame,f'{labelBox}  {conf}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,0,255),thickness=1)
            else:
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 1)
                cv2.putText(frame,f'{labelBox}  {conf}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,255,0),thickness=1)
    
    new_frame_time = time.time() 
    print(f'{round(new_frame_time-prevTime,2)} s')
    prevTime=new_frame_time
    cv2.imshow("Image",frame)
    if cv2.waitKey(10) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

0: 512x352 1 not_helmet, 1 not_safetyVest, 48.0ms
Speed: 0.0ms preprocess, 48.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 352)
1724778936.8 s

0: 512x352 1 not_helmet, 1 not_safetyVest, 32.0ms
Speed: 8.0ms preprocess, 32.0ms inference, 8.0ms postprocess per image at shape (1, 3, 512, 352)
0.09 s

0: 512x352 1 not_helmet, 1 not_safetyVest, 72.0ms
Speed: 0.0ms preprocess, 72.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 352)
0.12 s

0: 512x352 1 not_helmet, 1 not_safetyVest, 33.4ms
Speed: 8.0ms preprocess, 33.4ms inference, 4.2ms postprocess per image at shape (1, 3, 512, 352)
0.06 s

0: 512x352 1 not_helmet, 1 not_safetyVest, 47.9ms
Speed: 0.0ms preprocess, 47.9ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 352)
0.09 s

0: 512x352 1 not_helmet, 1 not_safetyVest, 64.2ms
Speed: 0.0ms preprocess, 64.2ms inference, 8.0ms postprocess per image at shape (1, 3, 512, 352)
0.14 s

0: 512x352 1 not_helmet, 1 not_safetyVest, 64.0ms
Speed: 8.0ms

#Run Model With Face Recognition Library

In [6]:
listIDImagesPath=[f for f in os.listdir('D:\Desktop\Study\AIP\ID_Worker') if f.endswith(('.jpg', '.jpeg', '.png'))]

In [7]:
listIDImagesPath

['ID1.jpg',
 'ID10.jpg',
 'ID2.jpg',
 'ID3.jpg',
 'ID4.jpg',
 'ID5.jpg',
 'ID6.jpg',
 'ID7.jpg',
 'ID8.jpg',
 'ID9.jpg']

In [8]:
encodeID=[]
IDs=[]
for path in listIDImagesPath:
    imgPath='D:/Desktop/Study/AIP/ID_Worker/'+path
    imageID=cv2.imread(imgPath)
    imageID=cv2.cvtColor(imageID,cv2.COLOR_BGR2RGB)
    #imageID=cv2.resize(imageID,(512,512))
    facelocate=face_recognition.face_locations(imageID)
    if len(facelocate)>0:
        encode=face_recognition.face_encodings(imageID,facelocate)[0]
        IDs.append( path[:-4])
        encodeID.append(encode)

In [9]:
IDs

['ID1', 'ID10', 'ID2', 'ID3', 'ID4', 'ID5', 'ID7', 'ID8']

In [10]:
import time 
prevTime=0
cap=cv2.VideoCapture(0)
while True:
    ret, frame= cap.read()
    frame=cv2.flip(frame,1)
    frame=cv2.resize(frame,(512,512))
    results=modelSegment.predict(source=frame,conf=0.6, save=False)
    for i in range(0,len(results[0])):
        #cv2_imshow((results[0].masks.data[i].numpy()*255).astype("uint8"))
        mask=results[0].masks.cpu().data[i].numpy()
        mask_image = frame.copy()
        mask_image[mask == 0] = [255, 255, 255]
        color_img=frame.copy()
        color_img[mask == 0] = [255, 255, 255]
        color_img[mask == 1] = [0, 255, 0]
        frame[mask == 1]=cv2.addWeighted(frame[mask == 1],0.5,color_img[mask == 1],0.5,0)

        facelocate=face_recognition.face_locations(mask_image)
        if len(facelocate)>0 :
            encode=face_recognition.face_encodings(mask_image,facelocate)
            for encodeFace, faceLoc in zip(encode,facelocate):
                matches = face_recognition.compare_faces(encodeID,encodeFace)
                faceDis = face_recognition.face_distance(encodeID,encodeFace)
                matchIndex=np.argmin(faceDis)
                print("matches",matchIndex)
                print('dis',faceDis[matchIndex])

                y1,x2,y2,x1 = facelocate[0]
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
                cv2.putText(frame,f'{IDs[matchIndex]}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,0,255),thickness=1)
        
        resDetect= modelDetect.predict(mask_image,conf=0.5)
        for r in resDetect:
            boxes=r.boxes
            for box in boxes:
              x1,y1,x2,y2= box.xyxy[0]
              x1,y1,x2,y2= int(x1),int(y1),int(x2),int(y2)
              w,h=x1-x2,y1-y2
              conf=ceil(float(box.conf[0])*100)/100
              labelBox=modelDetect.names.get(box.cls.item())
              if(labelBox=='not_helmet' or labelBox =='not_safetyVest'):
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
                cv2.putText(frame,f'{labelBox}  {conf}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,0,255),thickness=1)
              else:
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 1)
                cv2.putText(frame,f'{labelBox}  {conf}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,255,0),thickness=1)
        
    new_frame_time = time.time() 
    print(f'{round(new_frame_time-prevTime,2)} s')
    prevTime=new_frame_time
    cv2.imshow("Image",frame)
    if cv2.waitKey(10) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 512x512 1 people, 15.5ms
Speed: 2.0ms preprocess, 15.5ms inference, 84.3ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 not_helmet, 1 safetyVest, 17.7ms
Speed: 1.0ms preprocess, 17.7ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)
1723996258.18 s

0: 512x512 1 people, 11.5ms
Speed: 1.0ms preprocess, 11.5ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 not_helmet, 1 safetyVest, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)
0.28 s

0: 512x512 1 people, 11.1ms
Speed: 1.0ms preprocess, 11.1ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 not_helmet, 1 safetyVest, 16.0ms
Speed: 1.0ms preprocess, 16.0ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)
0.25 s

0: 512x512 1 people, 10.7ms
Speed: 1.0ms preprocess, 10.7ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 not_helmet, 1 safet

In [8]:
import time 
prevTime=0
cap=cv2.VideoCapture("D:/Games/Old vs. New Construction Worker.mp4")
while True:
    ret, frame= cap.read()
    if not ret:
        print("No more frames in the video")
        break
    frame=cv2.flip(frame,1)
    frame=cv2.resize(frame,(512,512))
    results=modelSegment.predict(source=frame,conf=0.6, save=False)
    for i in range(0,len(results[0])):
        #cv2_imshow((results[0].masks.data[i].numpy()*255).astype("uint8"))
        mask=results[0].masks.cpu().data[i].numpy()
        mask_image = frame.copy()
        mask_image[mask == 0] = [255, 255, 255]
        color_img=frame.copy()
        color_img[mask == 0] = [255, 255, 255]
        color_img[mask == 1] = [0, 255, 0]
        frame[mask == 1]=cv2.addWeighted(frame[mask == 1],0.5,color_img[mask == 1],0.5,0)

        facelocate=face_recognition.face_locations(mask_image)
        if len(facelocate)>0 :
            encode=face_recognition.face_encodings(mask_image,facelocate)
            for encodeFace, faceLoc in zip(encode,facelocate):
                matches = face_recognition.compare_faces(encodeID,encodeFace)
                faceDis = face_recognition.face_distance(encodeID,encodeFace)
                matchIndex=np.argmin(faceDis)
                print("matches",matchIndex)
                print('dis',faceDis[matchIndex])

                y1,x2,y2,x1 = facelocate[0]
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
                cv2.putText(frame,f'{IDs[matchIndex]}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,0,255),thickness=1)
        
        resDetect= modelDetect.predict(mask_image,conf=0.5)
        for r in resDetect:
            boxes=r.boxes
            for box in boxes:
              x1,y1,x2,y2= box.xyxy[0]
              x1,y1,x2,y2= int(x1),int(y1),int(x2),int(y2)
              w,h=x1-x2,y1-y2
              conf=ceil(float(box.conf[0])*100)/100
              labelBox=modelDetect.names.get(box.cls.item())
              if(labelBox=='not_helmet' or labelBox =='not_safetyVest'):
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
                cv2.putText(frame,f'{labelBox}  {conf}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,0,255),thickness=1)
              else:
                frame=cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 1)
                cv2.putText(frame,f'{labelBox}  {conf}',(x1,y1),cv2.FONT_HERSHEY_DUPLEX,fontScale=0.5,color=(0,255,0),thickness=1)
        
    new_frame_time = time.time() 
    print(f'{round(new_frame_time-prevTime,2)} s')
    prevTime=new_frame_time
    cv2.imshow("Image",frame)
    if cv2.waitKey(10) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 512x512 (no detections), 10.8ms
Speed: 3.1ms preprocess, 10.8ms inference, 18.2ms postprocess per image at shape (1, 3, 512, 512)
1722368442.14 s

0: 512x512 (no detections), 15.4ms
Speed: 1.5ms preprocess, 15.4ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)
0.12 s

0: 512x512 (no detections), 22.7ms
Speed: 2.5ms preprocess, 22.7ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)
0.05 s

0: 512x512 (no detections), 25.1ms
Speed: 2.0ms preprocess, 25.1ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)
0.06 s

0: 512x512 (no detections), 17.3ms
Speed: 2.6ms preprocess, 17.3ms inference, 2.5ms postprocess per image at shape (1, 3, 512, 512)
0.04 s

0: 512x512 (no detections), 14.7ms
Speed: 2.0ms preprocess, 14.7ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 512)
0.07 s

0: 512x512 1 people, 18.2ms
Speed: 2.0ms preprocess, 18.2ms inference, 88.2ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 helmet, 1